In [13]:
import numpy as np 
import djlib as dj
import pickle
import os

In [14]:
#User-specific paths
datafile = '/home/derick/experiments/stan_demo_ZrN/full_enumeration.json'
eci_posterior_file = 'results.pkl'


In [15]:
#Load casm query data and stan ECI data
data = dj.casm_query_reader(datafile)
with open(eci_posterior_file,'rb') as f:
    eci_posterior = pickle.load(f)
    eci_posterior = eci_posterior['eci']


In [16]:
#Format casm query data as numpy arrays, remove redundant dimensionality
corr = np.squeeze(np.array(data['corr']))
comp = np.reshape(np.array(data['comp']),(-1,1))
formation_energy = np.array(data['formation_energy'])
name = data['name']

In [17]:
#Examine shape of correlation matrix and sampled ECI
print(np.array(corr).shape)
print(np.array(eci_posterior).shape)

(67536, 253)
(253, 10000)


In [18]:
#Find the proposed ground state indices, write to file. This is time consuming, so do not run this if the proposed ground states file already exists. 
if os.path.isfile('proposed_ground_state_indices.pkl') == False:
    proposed_ground_state_indices = dj.clex.find_proposed_ground_states(corr, np.array(comp), np.array(formation_energy), eci_posterior.T)
    with open('proposed_ground_state_indices.pkl', 'wb') as f:
        pickle.dump({'indices':proposed_ground_state_indices}, f, protocol=pickle.HIGHEST_PROTOCOL)
else:
    print('Proposed ground states file exists.')


Proposed ground states file exists.


In [19]:
with open('proposed_ground_state_indices.pkl', 'rb') as f:
    proposed_ground_state_indices = pickle.load(f)
    proposed_ground_state_indices = proposed_ground_state_indices['indices']

In [20]:
#Quantify the new ground states that are appearing:
tally = np.bincount(proposed_ground_state_indices.astype(int))
unique_proposed_configs = np.unique(proposed_ground_state_indices.astype(int))

#Display statistics
config_percentage_dict = {}
config_percentage = []
proposed_names = []
for index, config in enumerate(unique_proposed_configs):
    print(name[config], end=' ')
    print('appears in %d of %d sample runs (%.6f percent)' % (tally[config], eci_posterior.shape[1], (100*tally[config]/eci_posterior.shape[1])) )
    config_percentage_dict["%s" % name[config]] = (100*tally[config]/eci_posterior.shape[1])
    config_percentage.append([config,(100*tally[config]/eci_posterior.shape[1])])
    proposed_names.append(name[config])

SCEL3_3_1_1_0_2_1/0 appears in 180 of 10000 sample runs (1.800000 percent)
SCEL4_2_2_1_0_1_0/0 appears in 6 of 10000 sample runs (0.060000 percent)
SCEL4_4_1_1_0_0_2/0 appears in 8001 of 10000 sample runs (80.010000 percent)
SCEL5_5_1_1_0_1_3/0 appears in 1274 of 10000 sample runs (12.740000 percent)
SCEL6_3_2_1_0_1_2/2 appears in 1932 of 10000 sample runs (19.320000 percent)
SCEL6_6_1_1_0_1_4/1 appears in 170 of 10000 sample runs (1.700000 percent)
SCEL6_6_1_1_0_3_2/7 appears in 1 of 10000 sample runs (0.010000 percent)
SCEL7_7_1_1_0_2_4/11 appears in 1 of 10000 sample runs (0.010000 percent)
SCEL7_7_1_1_0_3_5/0 appears in 2757 of 10000 sample runs (27.570000 percent)
SCEL7_7_1_1_0_3_5/1 appears in 164 of 10000 sample runs (1.640000 percent)
SCEL8_2_4_1_0_1_1/4 appears in 1294 of 10000 sample runs (12.940000 percent)
SCEL8_2_4_1_1_0_1/0 appears in 653 of 10000 sample runs (6.530000 percent)
SCEL8_2_4_1_1_0_1/1 appears in 4392 of 10000 sample runs (43.920000 percent)
SCEL8_4_1_2_0_0_1/

Proposed ground states are configurations that are below the cluster expansion prediction of the DFT hull for a given ECI set. Some configurations appear more frequently than others. It is possble to quantigy the frequency of these ground states by a percentage.

In [21]:
config_percentage = np.array(config_percentage)

In [22]:
#Filter configurations of interest by only viewing configs that appear some percentage of the time
percentile = config_percentage[:,-1] > 75
config_percentage[percentile].shape
proposed_names = np.array(proposed_names)
proposed_names[percentile]

array(['SCEL4_4_1_1_0_0_2/0', 'SCEL8_4_1_2_0_1_2/4',
       'SCEL8_8_1_1_0_5_2/10', 'SCEL10_5_1_2_0_2_3/7',
       'SCEL11_11_1_1_0_5_10/39', 'SCEL12_3_2_2_0_1_2/8',
       'SCEL12_3_4_1_2_1_0/23', 'SCEL12_4_3_1_1_0_0/9',
       'SCEL12_4_3_1_1_0_0/21', 'SCEL12_12_1_1_0_6_8/9',
       'SCEL12_12_1_1_0_9_3/9', 'SCEL14_7_1_2_0_6_3/26',
       'SCEL14_7_1_2_0_6_3/73', 'SCEL14_7_1_2_0_6_5/68',
       'SCEL14_7_1_2_0_6_5/73', 'SCEL15_15_1_1_0_0_13/391'], dtype='<U25')